## Exercising Ethereum traces streaming directly into BigQuery

Based on:
- https://github.com/blockchain-etl/blockchain-etl-streaming/tree/master
- https://medium.com/google-cloud/live-ethereum-and-bitcoin-data-in-google-bigquery-and-pub-sub-765b71cd57b5
- https://medium.com/google-cloud/real-time-ethereum-notifications-for-everyone-for-free-a76e72e45026

### Create a table to host Pub/Sub events ... Change dataset name

In [2]:
%%bigquery

create or replace table `genai-embeddings.yugalabs_bqmlga4.etherium_traces`
(
data JSON
);

Query is running:   0%|          |

""


### Create a view to unnest the JSON data ... Change dataset name

In [3]:
%%bigquery

create or replace view `genai-embeddings.yugalabs_bqmlga4.vw_etherium_traces` as
select
JSON_VALUE(data.block_hash) as block_hash,
CAST(JSON_VALUE(data.block_number) as INTEGER) as block_number,
CAST(JSON_VALUE(data.block_timestamp) as INTEGER) as block_timestamp,
JSON_VALUE(data.call_type) as call_type,
JSON_VALUE(data.error) as error,
JSON_VALUE( data.from_address) as from_address,
CAST(JSON_VALUE(data.gas) as INTEGER) as gas,
CAST(JSON_VALUE(data.gas_used) as INTEGER) as gas_used,
JSON_VALUE(data.input) as input,
JSON_VALUE(data.item_id) as item_id,
JSON_VALUE(data.item_timestamp) as item_timestamp,
JSON_VALUE(data.output) as output,
JSON_VALUE(data.reward_type) as reward_type,
CAST(JSON_VALUE(data.status) as INTEGER) as status,
CAST(JSON_VALUE(data.subtraces) as INTEGER) as subtraces,
JSON_VALUE(data.to_address) as to_address,
JSON_VALUE(data.trace_address) as trace_address,
JSON_VALUE(data.trace_id) as trace_id,
CAST(JSON_VALUE(data.trace_index) as INTEGER) as trace_index,
JSON_VALUE(data.trace_type) as trace_type,
JSON_VALUE(data.transaction_hash) as transaction_hash,
CAST(JSON_VALUE(data.transaction_index) as INTEGER) as transaction_index,
JSON_VALUE(data.type) as type,
CAST(JSON_VALUE(data.value) as INTEGER) as value
from `genai-embeddings.yugalabs_bqmlga4.etherium_traces`


Query is running:   0%|          |

""


### Create a Pub/Sub Subscription to the Public Topic named crypto_ethereum.traces under project crypto-public-data

In [4]:
! gcloud pubsub subscriptions create crypto_ethereum.traces --topic=crypto_ethereum.traces --topic-project=crypto-public-data

Created subscription [projects/genai-embeddings/subscriptions/crypto_ethereum.traces].


To take a quick anonymous survey, run:
  $ gcloud survey



### Read a single message to test if it works

In [5]:
! gcloud pubsub subscriptions pull crypto_ethereum.traces

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Delete Subscription as we will now recreate the subscription pointing directly to the BigQuery table created earlier

In [14]:
! gcloud pubsub subscriptions delete crypto_ethereum.traces

Deleted subscription [projects/genai-embeddings/subscriptions/crypto_ethereum.traces].


### Using the GCP Console, assign BigQuery role (BQ Data Editor) to the Pub/Sub service account
- The Pub/Sub service account has this naming convention: service-"your-project-number"@gcp-sa-pubsub.iam.gserviceaccount.com

### Create a Pub/Sub Subscription pointing it to the BigQuery table created earlier
-- More info and options at https://cloud.google.com/pubsub/docs/create-bigquery-subscription#before_you_begin

In [10]:
! gcloud pubsub subscriptions create crypto_ethereum.traces \
    --topic=crypto_ethereum.traces \
    --topic-project=crypto-public-data \
    --bigquery-table=genai-embeddings:yugalabs_bqmlga4.etherium_traces

Created subscription [projects/genai-embeddings/subscriptions/crypto_ethereum.traces].


### Validate if data is being added to the table

In [ ]:
%%bigquery
select * from yugalabs_bqmlga4.etherium_traces LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,data
0,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
1,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
2,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
3,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
4,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
5,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
6,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
7,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
8,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."
9,"{""block_hash"":""0xa3585ea87a8659e187100c8ab1a34..."


### Now check the view

In [16]:
%%bigquery
select * from yugalabs_bqmlga4.vw_etherium_traces LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,block_hash,block_number,block_timestamp,call_type,error,from_address,gas,gas_used,input,item_id,...,subtraces,to_address,trace_address,trace_id,trace_index,trace_type,transaction_hash,transaction_index,type,value
0,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0,250371,197305,0xc9e75c48000000000000000031010000000000000000...,trace_call_0x613d806638785232065a36a0c48f043ec...,...,2,0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0,None,call_0x613d806638785232065a36a0c48f043eca0bc33...,3,call,0x613d806638785232065a36a0c48f043eca0bc3395475...,0,trace,0
1,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,staticcall,None,0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0,106209,2517,0x0902f1ac,trace_call_0x613d806638785232065a36a0c48f043ec...,...,0,0x6a091a3406e0073c3cd6340122143009adac0eda,None,call_0x613d806638785232065a36a0c48f043eca0bc33...,17,call,0x613d806638785232065a36a0c48f043eca0bc3395475...,0,trace,0
2,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,staticcall,None,0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0,103512,646,0x70a082310000000000000000000000006a091a3406e0...,trace_call_0x613d806638785232065a36a0c48f043ec...,...,0,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,None,call_0x613d806638785232065a36a0c48f043eca0bc33...,18,call,0x613d806638785232065a36a0c48f043eca0bc3395475...,0,trace,0
3,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,staticcall,None,0x6a091a3406e0073c3cd6340122143009adac0eda,238528,534,0x70a082310000000000000000000000006a091a3406e0...,trace_call_0x74e8aeaba0f705f72f8df1312d51ded3b...,...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,call_0x74e8aeaba0f705f72f8df1312d51ded3b715986...,35,call,0x74e8aeaba0f705f72f8df1312d51ded3b715986c9d6f...,1,trace,0
4,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x7b577a879665ef6ad344da61de7ec2371f6ff68f,2300,0,0x,trace_call_0x74e8aeaba0f705f72f8df1312d51ded3b...,...,0,0x1f9090aae28b8a3dceadf281b0f12828e676c326,None,call_0x74e8aeaba0f705f72f8df1312d51ded3b715986...,36,call,0x74e8aeaba0f705f72f8df1312d51ded3b715986c9d6f...,1,trace,256016394959325000
5,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x692452d0a61792e01e98b25aac0f49910b72bd92,245508,164209,0x791ac947000000000000000000000000000000000000...,trace_call_0xccdf5ce880c6688b1b876af9cdbba9302...,...,7,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,None,call_0xccdf5ce880c6688b1b876af9cdbba930214985a...,37,call,0xccdf5ce880c6688b1b876af9cdbba930214985aa6062...,2,trace,0
6,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,237075,67252,0x23b872dd000000000000000000000000692452d0a617...,trace_call_0xccdf5ce880c6688b1b876af9cdbba9302...,...,0,0x642ac912a58428767fa14a26a749f9a1b001ba92,None,call_0xccdf5ce880c6688b1b876af9cdbba930214985a...,38,call,0xccdf5ce880c6688b1b876af9cdbba930214985aa6062...,2,trace,0
7,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x00000047bb99ea4d791bb749d970de71ee0b1a34,142361,10524,0x2e1a7d4d000000000000000000000000000000000000...,trace_call_0x5db9008cd625c5c4d545a3c9273f90a83...,...,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,call_0x5db9008cd625c5c4d545a3c9273f90a834c03f7...,63,call,0x5db9008cd625c5c4d545a3c9273f90a834c03f7f459f...,3,trace,0
8,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2300,1384,0x,trace_call_0x5db9008cd625c5c4d545a3c9273f90a83...,...,0,0x00000047bb99ea4d791bb749d970de71ee0b1a34,None,call_0x5db9008cd625c5c4d545a3c9273f90a834c03f7...,64,call,0x5db9008cd625c5c4d545a3c9273f90a834c03f7f459f...,3,trace,818148350618342913
9,0xa3585ea87a8659e187100c8ab1a34127971fe1026f41...,18214997,1695671447,call,None,0x00000047bb99ea4d791bb749d970de71ee0b1a34,124786,0,0x,trace_call_0x5db9008cd625c5c4d545a3c9273f90a83...,...